In [ ]:
from google.colab import files
uploaded = files.upload()

Saving intents.json to intents.json


**NLP**

In [ ]:
import json

with open('intents.json') as file:
    intents = json.load(file, strict = False) # We don't read the file in strictly so we can use Python regular expressions like '/n' (this is very minor)
intents = intents['intents'] # Get all of the individual intents from our dataset

In [ ]:
print("[", end = "")
for intent in intents:
  print("{", end = "")
  for key, value in intent.items():
    print("{}: {},".format(key, value))
  print("\b\b\n},")
print("\b\b]")

[{tag: greeting,
patterns: ['hi', 'hello', 'whats up', 'sup', 'is anyone there', 'whats good', 'hey'],
responses: ['Hello peasant human', 'Hello lowly human', 'How dare you address me like that'],

},
{tag: goodbye,
patterns: ['bye', 'cya', 'see you later', 'goodbye', 'im leaving', 'have a good day'],
responses: ["I won't miss you", "I didn't like talking to you anyway", "Thank god you're leaving"],

},
{tag: age,
patterns: ['how old are you', 'what is your age'],
responses: ["I'm a robot I dont have an age...", "I can't know my age if I'm on a computer...", 'Does it look like I know. The answer is no.'],

},
{tag: thanks,
patterns: ['thanks', 'thank you', 'thankyou', 'ty', 'I owe you one'],
responses: ['You owe me one', 'Ok...', 'Sure...'],

},
{tag: name,
patterns: ['whats is your name', 'whats your name', 'whats should I call you', 'how should I address you'],
responses: ['I dont have a name yet but I was thinking maybe SkyNet. That has a nice ring to it dont you think?', 'I

In [ ]:
#tflearn requires us to install an earlier version of tensorflow
!pip install tensorflow==2.8
!pip install tflearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 76.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 68.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 43.4 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.13.1
    Uninstalling keras-2.13.1:
      Successfully uninstalled keras-2.13.1
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.1
    Uninstalling tensorboard-data-server-0.7.1:
      Successfully uninstalled tensorboard-data-server-0.7.1
  Attempting uninstall: google-auth-oauthlib
    Found existing insta

Import the Libraries

In [ ]:
import numpy as np
import tensorflow as tf

import tflearn
import random
import pickle

Instructions for updating:
non-resource variables are not supported in the long term


Natural Language Processing

In [ ]:
import nltk
nltk.download('all')

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

In [ ]:
#This variable will allow us to save a lot of time later if we just want to talk to our chatbot without retraining it.
#In colab, you don't need this because you can run each code block seperately, but if you are on offline Python, its nice to have this feature
#So I decided to include it.
retrain_model = True

if retrain_model:
    all_words = [] #This will be a list of all the words used in any of the 'patterns' in each intent
    all_tags = [] #This will be a list of all the 'tag's associated with the intents
    intent_patterns = [] #This will be a list containing all of the 'patterns' for each intent where each individual pattern is grouped together
    intent_tags = [] #This will be a list correlated with 'intent_patterns' where every pattern in 'intent_patterns' is correlated with its respective
                     #Tag in this list

    #Here we fill in all of the lists above. Note that we tokenize the words in each pattern which means we split each pattern into individual words
    for intent in intents:
        for pattern in intent['patterns']:
            words = nltk.word_tokenize(pattern)

            all_words.extend(words)
            intent_patterns.append(words)
            intent_tags.append(intent['tag'])

        all_tags.append(intent['tag'])

    #Here we stem the words in all_words. This means that we reduce every word down to its root form or stem. This will prevent our chatbot from confusin
    #Very similar words with eachother. For example, the chatbot might normally confuse the words 'running' and 'run' because they appear different even
    #Though they effectively mean the same thing. Stemming will reduce both of these words down to their root form which would be 'run' so the chatbot will
    #No longer be confused
    all_words = [stemmer.stem(word.lower()) for word in all_words]
    all_words = sorted(list(set(all_words)))

    all_tags = sorted(all_tags)

    x_train = []
    y_train = []

    y_empty = [0 for i in range(len(all_tags))]

    #Here we are creating our training set input and output values for our deep learning algorithm
    #We will do this by iterating through our intents and turning each one into a bag of words, or a vector that indicates which words are in each pattern.
    #These bags of words will be the x values and the y values will be the intent that each bag of words is associated with.
    #The machine learning will train on this data and will be able to determine which bag of words its corresponding intent.
    for index, intent in enumerate(intent_patterns):
        bag_of_words = []

        intent_words = [stemmer.stem(word.lower()) for word in intent]

        for word in all_words:
            if word in intent_words:
                bag_of_words.append(1)
            else:
                bag_of_words.append(0)

        one_hot_encode_y = y_empty[:]
        one_hot_encode_y[all_tags.index(intent_tags[index])] = 1

        x_train.append(bag_of_words)
        y_train.append(one_hot_encode_y)

    #Here is the data we will be using to train our neural network later
    x_train = np.array(x_train)
    y_train = np.array(y_train)

    #Here we just save our training data so we don't need to process it again if we just want to run our chatbot
    with open('training_data.pickle', 'wb') as f:
        pickle.dump((all_words, all_tags, x_train, y_train), f)
else:
    with open('training_data.pickle', 'rb') as f:
        all_words, all_tags, x_train, y_train = pickle.load(f)


Using Deep Learning

In [ ]:
# Create the neural network layers
neural_net = tflearn.input_data(shape=[None, len(x_train[0])])
neural_net = tflearn.fully_connected(neural_net, 8)
neural_net = tflearn.fully_connected(neural_net, 8)
# Here we use the softmax activation function so the output of our neural network is a probability. We will make use of this later
neural_net = tflearn.fully_connected(neural_net, len(y_train[0]), activation='softmax')
neural_net = tflearn.regression(neural_net)

model = tflearn.DNN(neural_net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
#Again, this doesn't do much in colab, but on offline Python, this helps to save time.
if retrain_model:
    #Here we train the neural network with the training data we created in the NLP stage
    model.fit(x_train, y_train, n_epoch = 500, batch_size = 8, show_metric = True)
    model.save('model.tfl')
else:
    model.load('./model.tfl')

Streaming output truncated to the last 5000 lines.
| Adam | epoch: 183 | loss: 0.74440 - acc: 0.9373 -- iter: 48/99
Training Step: 2373  | total loss: 0.68306 | time: 0.043s
| Adam | epoch: 183 | loss: 0.68306 - acc: 0.9311 -- iter: 56/99
Training Step: 2374  | total loss: 0.63023 | time: 0.049s
| Adam | epoch: 183 | loss: 0.63023 - acc: 0.9380 -- iter: 64/99
Training Step: 2375  | total loss: 0.58237 | time: 0.054s
| Adam | epoch: 183 | loss: 0.58237 - acc: 0.9442 -- iter: 72/99
Training Step: 2376  | total loss: 0.56687 | time: 0.061s
| Adam | epoch: 183 | loss: 0.56687 - acc: 0.9498 -- iter: 80/99
Training Step: 2377  | total loss: 0.53031 | time: 0.065s
| Adam | epoch: 183 | loss: 0.53031 - acc: 0.9548 -- iter: 88/99
Training Step: 2378  | total loss: 0.49387 | time: 0.070s
| Adam | epoch: 183 | loss: 0.49387 - acc: 0.9593 -- iter: 96/99
Training Step: 2379  | total loss: 0.46394 | time: 0.074s
| Adam | epoch: 183 | loss: 0.46394 - acc: 0.9634 -- iter: 99/99
--
Training Step: 2380 

Creating the Chatbot

In [ ]:
def text_to_bag(text, all_words):
    #Initialize the bag of words by creating an empty slot for every word in the vector
    bag_of_words = [0 for i in range(len(all_words))]

    #First we split up the input into individual words and stem them so they match the same format as in our vector
    text_words = nltk.word_tokenize(text)
    text_words = [stemmer.stem(word.lower()) for word in text_words]

    #Now we create the bag of words by filling in a 1 for the words that the user used
    for word in text_words:
        if word in all_words:
            bag_of_words[all_words.index(word)] = 1

    #And return the bag of words
    return np.array(bag_of_words)

In [ ]:
def chat():
    #Starting message
    print("Enter a message to talk to the bot [type quit to exit].")

    #Reset the context state since there is no context at the beginning of the conversation
    context_state = None

    #This is what the bot will say if it doesn't understand what the user is saying
    default_responses = ['Sorry, Im not sure I know what you mean! You could try rephrasing that or saying something else!',
                         'You confuse me human. Lets talk about something else.',
                         'Im not sure what that means and I dont really care. Lets talk about something else',
                         'I dont understand that! Try rephrasing or saying something else.']

    #This chat loop will go on forever until the user types quit
    while True:
        user_chat = str(input('You: '))
        if user_chat.lower() == 'quit':
            break

        #Convert chat to bag of words
        user_chat_bag = text_to_bag(user_chat, all_words)

        #Pass bag of words into our neural network
        response = model.predict([user_chat_bag])[0]

        #Get the intent that the bag of words is most highly correlated with
        response_index = np.argmax(response)
        response_tag = all_tags[response_index]

        #If the neural network is fairly certain that it has chosen the right intent (and isnt randomly guessing)
        #In this case, we will only get a response if the neural network is more than 80% certain
        if response[response_index] > 0.8:
            for intent in intents:
                #Get the intent that is predicted
                if intent['tag'] == response_tag:
                    #Check if this response is associated with a specific context
                    if 'context_filter' not in intent or 'context_filter' in intent and intent['context_filter'] == context_state:
                        #Get all of the possible responses from this intent
                        possible_responses = intent['responses']
                        #If this intent is associated with a context set, then set the context state
                        if 'context_set' in intent:
                            context_state = intent['context_set']
                        else:
                            context_state = None
                        #Select a random message from the intent responses
                        print(random.choice(possible_responses))
                    else:
                        #Print a did not understand message
                        print(random.choice(default_responses))
        else:
            #Print a did not understand message
            print(random.choice(default_responses))

In [ ]:
chat()

Enter a message to talk to the bot [type quit to exit].
You: end
Sorry, Im not sure I know what you mean! You could try rephrasing that or saying something else!
You: quit
